In [28]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 12: Leonardo's Monorail ---</h2><p>You finally reach the top floor of this building: a garden with a slanted glass ceiling. Looks like there are no more stars to be had.</p>
<p>While sitting on a nearby bench amidst some <a href="https://www.google.com/search?q=tiger+lilies&amp;tbm=isch">tiger lilies</a>, you manage to decrypt some of the files you extracted from the servers downstairs.</p>
<p>According to these documents, Easter Bunny HQ isn't just this building - it's a collection of buildings in the nearby area. They're all connected by a local monorail, and there's another building not far from here! Unfortunately, being night, the monorail is currently not operating.</p>
<p>You remotely connect to the monorail control systems and discover that the boot sequence expects a password. The password-checking logic (your puzzle input) is easy to extract, but the code it uses is strange: it's <span title="Strangely, this assembunny code doesn't seem to be very good at multiplying.">assembunny</span> code designed for the <a href="11">new computer</a> you just assembled. You'll have to execute the code and get the password.</p>
<p>The assembunny code you've extracted operates on four <a href="https://en.wikipedia.org/wiki/Processor_register">registers</a> (<code>a</code>, <code>b</code>, <code>c</code>, and <code>d</code>) that start at <code>0</code> and can hold any <a href="https://en.wikipedia.org/wiki/Integer">integer</a>. However, it seems to make use of only a few <a href="https://en.wikipedia.org/wiki/Instruction_set">instructions</a>:</p>
<ul>
<li><code>cpy x y</code> <em>copies</em> <code>x</code> (either an integer or the <em>value</em> of a register) into register <code>y</code>.</li>
<li><code>inc x</code> <em>increases</em> the value of register <code>x</code> by one.</li>
<li><code>dec x</code> <em>decreases</em> the value of register <code>x</code> by one.</li>
<li><code>jnz x y</code> <em>jumps</em> to an instruction <code>y</code> away (positive means forward; negative means backward), but only if <code>x</code> is <em>not zero</em>.</li>
</ul>
<p>The <code>jnz</code> instruction moves relative to itself: an offset of <code>-1</code> would continue at the previous instruction, while an offset of <code>2</code> would <em>skip over</em> the next instruction.</p>
<p>For example:</p>
<pre><code>cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a
</code></pre>
<p>The above code would set register <code>a</code> to <code>41</code>, increase its value by <code>2</code>, decrease its value by <code>1</code>, and then skip the last <code>dec a</code> (because <code>a</code> is not zero, so the <code>jnz a 2</code> skips it), leaving register <code>a</code> at <code>42</code>. When you move past the last instruction, the program halts.</p>
<p>After executing the assembunny code in your puzzle input, <em>what value is left in register <code>a</code>?</em></p>
</article>


In [36]:
from atexit import register
from tabulate import tabulate


program_example = """
cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a
"""


def process(
    program: str, register_names: str = "abcd", register_initializer=lambda x: 0
) -> dict[str, int]:
    instructions = [i.split() for i in program.strip().splitlines()]
    registers = {l: register_initializer(l) for l in register_names}
    print(tabulate(enumerate(instructions)))
    counter = 0

    while counter < len(instructions):
        instruction = instructions[counter]
        if instruction[0] == "cpy":
            # cpy x y copies x (either an integer or the value of a register)
            # into register y.
            _, x, y = instruction
            if x in register_names:
                registers[y] = registers[x]
            else:
                registers[y] = int(x)
            counter += 1
        elif instruction[0] == "inc":
            # inc x increases the value of register x by one.
            _, x = instruction
            registers[x] += 1
            counter += 1
        elif instruction[0] == "dec":
            # dec x decreases the value of register x by one.
            _, x = instruction
            registers[x] -= 1
            counter += 1
        elif instruction[0] == "jnz":
            # jnz x y jumps to an instruction y away
            # (positive means forward; negative means backward),
            # but only if x is not zero.
            _, x, y = instruction

            if x in register_names and registers[x] != 0:
                counter += int(y)
            elif x.isdecimal() and int(x) != 0:
                counter += int(y)
            else:
                counter += 1
        else:
            raise ValueError(f"Unkonw isntruction: {instruction}")

    return registers


example_answer = process(program_example)
assert example_answer["a"] == 42

-  ------------------
0  ['cpy', '41', 'a']
1  ['inc', 'a']
2  ['inc', 'a']
3  ['dec', 'a']
4  ['jnz', 'a', '2']
5  ['dec', 'a']
-  ------------------


In [37]:
with open("../input/day12.txt") as f:
    program_I = f.read()


process(program_I)

--  ------------------
 0  ['cpy', '1', 'a']
 1  ['cpy', '1', 'b']
 2  ['cpy', '26', 'd']
 3  ['jnz', 'c', '2']
 4  ['jnz', '1', '5']
 5  ['cpy', '7', 'c']
 6  ['inc', 'd']
 7  ['dec', 'c']
 8  ['jnz', 'c', '-2']
 9  ['cpy', 'a', 'c']
10  ['inc', 'a']
11  ['dec', 'b']
12  ['jnz', 'b', '-2']
13  ['cpy', 'c', 'b']
14  ['dec', 'd']
15  ['jnz', 'd', '-6']
16  ['cpy', '13', 'c']
17  ['cpy', '14', 'd']
18  ['inc', 'a']
19  ['dec', 'd']
20  ['jnz', 'd', '-2']
21  ['dec', 'c']
22  ['jnz', 'c', '-5']
--  ------------------


{'a': 317993, 'b': 196418, 'c': 0, 'd': 0}

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>317993</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you head down the fire escape to the monorail, you notice it didn't start; register <code>c</code> needs to be initialized to the position of the ignition key.</p>
<p>If you instead <em>initialize register <code>c</code> to be <code>1</code></em>, what value is now left in register <code>a</code>?</p>
</article>

</main>


In [39]:
process(program_I, register_initializer=lambda l: 0 if l != "c" else 1)

--  ------------------
 0  ['cpy', '1', 'a']
 1  ['cpy', '1', 'b']
 2  ['cpy', '26', 'd']
 3  ['jnz', 'c', '2']
 4  ['jnz', '1', '5']
 5  ['cpy', '7', 'c']
 6  ['inc', 'd']
 7  ['dec', 'c']
 8  ['jnz', 'c', '-2']
 9  ['cpy', 'a', 'c']
10  ['inc', 'a']
11  ['dec', 'b']
12  ['jnz', 'b', '-2']
13  ['cpy', 'c', 'b']
14  ['dec', 'd']
15  ['jnz', 'd', '-6']
16  ['cpy', '13', 'c']
17  ['cpy', '14', 'd']
18  ['inc', 'a']
19  ['dec', 'd']
20  ['jnz', 'd', '-2']
21  ['dec', 'c']
22  ['jnz', 'c', '-5']
--  ------------------


{'a': 9227647, 'b': 5702887, 'c': 0, 'd': 0}

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>9227647</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
